In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [6]:
from langchain_groq import ChatGroq
def multiply(a:int,b:int)->int:
    """Multiply a and b?
    
    Args:
        a : first int
        b:second int """
    return a*b  

def add(a:int , b:int)-> int:
    """add a and b.

    Args:
    a : first int
    b:  second int
    """
    return a + b


def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b


tools = [add,multiply,divide]
llm=ChatGroq(model="qwen-2.5-32b")

llm_with_tools = llm.bind_tools(tools,parallel_tool_calls=False)


In [7]:
from langgraph.graph import MessagesState
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [8]:
from langchain_core.messages import HumanMessage,SystemMessage

sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

def assistant(state:MessagesState):
    return {'messages':[llm_with_tools.invoke([sys_msg]+state['messages'])]}

In [ ]:
from langgraph.graph import START,StateGraph
from langgraph.graph import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image , display

builder = StateGraph(MessagesState)

builder.add_node("assistant",assistant)
builder.add_node('tools',ToolNode(tools))

builder.add_conditional_edges('assistant',tools_condition,)

builder.add_edge('tools','assistant')
